# Overview
More or less a clone of `bold_register_scratch.ipynb` only now we are going to rely on manually alignment instead of the always failing ML models from ANTs. I don't mean to entirely imply that it is ANTs' fault, I might be misusing the models. However, it's proving difficult, and iterating over something that takes so long to align is taking forever. So lets go with the manual alignment.

**steps**
We will load all the data, run motion correction, and then manually align

# Setup and Load Data

In [1]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact
import pickle
import napari
import sparse

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"
cache_folder = "/Volumes/GlyphA_R1/nvol_cache"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

project root: /Users/joachimpfefferkorn/repos/neurovolume


In [2]:
bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)

# Functions

In [3]:
#TODO
#replace with napari
def compare_bold_alignment_legacy(bold_seq_vol: np.ndarray, anat_vol: np.ndarray, dim='x'):
    def x_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('x axis view')

        axes[0].imshow(bold_seq_vol[slice_idx,:,:, frame_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[slice_idx,:,:], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[slice_idx,:,:], cmap='gray')
        axes[2].imshow(bold_seq_vol[slice_idx,:,:, frame_idx], cmap='hot', alpha=opacity)
        axes[2].set_title('Overlay')

    def y_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('y axis view')

        axes[0].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,slice_idx,:], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,slice_idx,:], cmap='gray')
        axes[2].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap='hot', alpha=opacity)
        axes[2].set_title('Overlay')

    def z_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('z axis view')

        axes[0].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,:,slice_idx], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,:,slice_idx], cmap='gray')
        axes[2].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap='hot', alpha=opacity)
        axes[2].set_title('Overlay')

    match dim:
        case "x":
            interact(x_coord, slice_idx=(0, anat_vol.shape[0]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))
        case 'y':
            interact(y_coord, slice_idx=(0, anat_vol.shape[1]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))
        case 'z':
            interact(z_coord, slice_idx=(0, anat_vol.shape[2]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))


# The Meat of it All

In [4]:
# Truncated Version
sliced = bold_image.numpy()[:, :, :, :10]
bold_truncated_img = ants.from_numpy(sliced, spacing=bold_image.spacing, origin=bold_image.origin, direction=bold_image.direction)
#stabilized = ants.motion_correction(bold_image)
stabilized_truncated = ants.motion_correction(bold_truncated_img)

In [5]:
print(bold_truncated_img.numpy().shape)
print(bold_truncated_img.numpy().dtype)
print(t1_image.numpy().dtype)
print(t1_image.numpy().shape[0])

(64, 64, 35, 10)
float32
float32
512


In [6]:
def align_stabilized_bold_to_anat_p1(bold_img, t1_img, template_frame_idx=0, cache_dir=cache_folder):
    """"
    This function aligns a 4D BOLD image to a T1 anatomy image
    by aligning the mean of the BOLD to the T1 anatomy.
    It uses frame registration from only one "template frame"
    as we are assuming you're using a motion corrected
    BOLD image (or something relatively stable) as your input
    
    It gets things in the ballpark, but does will require some
    extra manual registration afterwards.
    """

    #This should also work with a mean image, I wonder what's better? TODO figure out this question
    print("Creating template frame")
    template_frame_idx = ants.from_numpy(bold_img.numpy()[:,:,:,template_frame_idx], spacing=bold_img.spacing[:3])
    frame_registration = ants.registration(
        fixed=t1_img,
        moving=template_frame_idx,
        type_of_transform="Rigid", 
    )
    output_shape = (t1_image.numpy().shape[0], t1_image.numpy().shape[1], t1_image.numpy().shape[2], bold_img.numpy().shape[3])
    registered_frames = np.zeros(shape=output_shape, dtype=bold_image.dtype)

    for frame in range(bold_img.shape[3]):
        print(f"aligning frame {frame + 1}/{bold_img.shape[3]} ")
        bold_frame = ants.from_numpy(bold_img.numpy()[:,:,:,frame],
                                    spacing=bold_img.spacing[:3])
        registered_frame_data = sparse.COO.from_numpy(ants.apply_transforms(
            fixed=t1_img,
            moving=bold_frame,
            transformlist=frame_registration['fwdtransforms'],
            interpolator='linear'
        ).numpy())
        registered_frames[:,:,:,frame] = registered_frame_data
    #sparse_registered_frames = sparse.COO.from_numpy(registered_frames) #Honestly we might not even need sparse and it crashed right at this moment
    #Lets just use the numpy for now and assess the memory load
    print("done")
    return registered_frames

#breaking this up into multiple functions for testing

Registering just the truncated bold back to ANTS was taking upwards of 20 minutes. Upon reflection, I began to wonder if it was even necessary? We're casting this to numpy arrays anyways, why go back to ANTS?

In [7]:
registered_BOLD = align_stabilized_bold_to_anat_p1(stabilized_truncated['motion_corrected'], t1_image) #actually not sparse, whoops!

Creating template frame
aligning frame 1/10 
aligning frame 2/10 
aligning frame 3/10 
aligning frame 4/10 
aligning frame 5/10 
aligning frame 6/10 
aligning frame 7/10 
aligning frame 8/10 
aligning frame 9/10 
aligning frame 10/10 
done


In [8]:
transposed_registered_bold = np.transpose(registered_BOLD, axes=(3,0,1,2))

In [9]:
print(registered_BOLD.shape)
print(transposed_registered_bold.shape)

(512, 512, 296, 10)
(10, 512, 512, 296)


In [10]:
viewer = napari.Viewer()

2024-12-06 17:51:09.449 Python[84381:3127298] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [14]:
viewer.add_image(transposed_registered_bold)
viewer.add_image(t1_image.numpy())

<Image layer 'Image' at 0x37613dad0>

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File ~/repos/neurovolume/.venv/lib/python3.11/site-packages/napari/_qt/threads/status_checker.py:100, in StatusChecker.calculate_status(self=<napari._qt.threads.status_checker.StatusChecker object>)
     97     return
     99 try:
--> 100     res = viewer._calc_status_from_cursor()
        viewer = Viewer(camera=Camera(center=(255.5, 255.5, 147.5), zoom=np.float64(1.5938476562499997), angles=(np.float64(80.68049771468225), np.float64(0.04245501672671794), np.float64(119.42796003915873)), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(np.float64(5.0), np.float64(256.3866496405162), np.float64(256.8675941035636), np.float64(282.8754390206965)), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=4, ndisplay=3, order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3'), rollable=(True, Tru

Traceback (most recent call last):
  File "/Users/joachimpfefferkorn/repos/neurovolume/.venv/lib/python3.11/site-packages/napari/_qt/threads/status_checker.py", line 86, in run
    self.calculate_status()
  File "/Users/joachimpfefferkorn/repos/neurovolume/.venv/lib/python3.11/site-packages/napari/_qt/threads/status_checker.py", line 110, in calculate_status
    self.status_and_tooltip_changed.emit(res)
                                         ^^^
UnboundLocalError: cannot access local variable 'res' where it is not associated with a value


Napari can't smoothly handle a truncated `(10, 512, 512, 296)` registered BOLD image, let alone an entire `185` sequence.

However, it is proving to be an incredibly valuable tool in checking anatomical alignment.

I believe that if I can transform our BOLD matrices in Napari the best workflow is the following:

- Skull Strip T1
- Skull Strip Bold (if possible/applicable)
- Register skull stripped T1 and Bold
- Manually align based off a template frame (either the generated mean or a chosen frame) with Napari
- Apply alignment to non-skull stripped versions
- Add all four grids -skull stripped T1, Full T1, Skull stripped BOLD, Full Bold- to `.nervol`